Laboration för kursen DD1334
Använder SQL-lite för att kommunicera med en databas som sparar data från vårt projekt i kartgenerering

Först måste vi hämta hem map klasser från github och installera dependencies

In [ ]:
! pip install depend #fixa sen
%pip install ipywidgets
! git clone git@github.com:poskusen/dd1334.git

Vi börjar med att generera en karta som ska sparas.

In [69]:
from Classes.map import Map
import ipywidgets as widgets
from IPython.display import display, clear_output
from draw_PNG import draw_map
ny_karta = Map(5, 100, 0, 0 ,100, realm_name= 'test_karta') # Skapa en karta med bara byar
draw_map(ny_karta)
output = widgets.Output()
with output: # Visa upp kartan
    image = open("test_image.png", "rb").read() 
    
    image_widget = widgets.Image(
        value=image,
        format='png',
        width=1000,            
        height=1000
    )
    display(image_widget)
display(output)


Output()

Sen connectar vi till databasen och skapar ett nytt table för kartor, continenter och allt content på kontinenterna.

Här uppstår ett stort problem som vi undviker genom att använda pickle.
För att spara vår representation av kontinenten så måste vi spara en lista med punkter. Detta blir väldigt tricky då SQL databaser är baserade på att man sparar 1 värde och inte en lista med värden. Något vi undviker när vi bara serialiserar hela objektet. 
En lösning hade varit skapa ett table per lista och därmed en per kontinent, men det är väldigt bökigt och förstör hela poängen med databasen. Och eftersom vi aldrig vill ha enbart en eller flera punkter från kontinent-listan finns det ingen poäng med att skapa ett helt nytt table, då vi aldrig hämtar enskilda värden från listan. Därför kan man se listan snarare som ett objekt, än en lista. Därför serialiserar vi listan och sparar den sen, allt annat hade varit galenskap.

För att ändå visa att det går gör vi det för städerna.
Vi sparar även bara städerna för floder och berg hade inneburit dubbelarbete, och visar egentligen ingenting.

Den enda fördelen med att köra sql är att det går att hämta kontinenter med rätt 'size' från databasen, dvs att jämföra storleken på serilizationen av points_list.

In [38]:
import sqlite3, csv
conn = sqlite3.connect('database.db')
cursor = conn.cursor()
cursor.execute('PRAGMA foreign_keys = ON;')
try:
    cursor.execute('DROP TABLE maps')
    cursor.execute('DROP TABLE continents')
    cursor.execute('DROP TABLE villages')
except:
    pass

cursor.execute('''CREATE TABLE maps(
               id INT PRIMARY KEY,
               map_size_x INT DEFAULT 1000,
               map_size_y INT DEFAULT 1000,
               name VARCHAR(255) NOT NULL,
               amount_continents INT NOT NULL
               );''') # Skapa ett table för kartor

cursor.execute('''CREATE TABLE continents(
                id INT PRIMARY KEY,
                points_serialised VARCHAR(65535),
                name_map VARCHAR(255) NOT NULL,
                FOREIGN KEY (name_map) REFERENCES maps(name)
               );''') # Skapa ett table för kontintenter


cursor.execute('''CREATE TABLE villages(
                id INT PRIMARY KEY,
                x INT NOT NULL,
                y INT NOT NULL,
                id_cont INT NOT NULL,
                FOREIGN KEY (id_cont) REFERENCES continents(id)
               );''') # Skapa ett table för byar



Skapa en enkel serialiserings algoritm som kan spara listor och en som kan återskapa innehållet och göra det till en lista

In [59]:
def stringify(continent_list):
    ser = ''
    for point in continent_list:
        str_point = str(point)
        str_point = str_point.strip('()')
        str_point = str_point.replace(' ','')
        ser +=  str_point + ','
    return ser

    

def unstringify(ser_cont):
    continent_list = []
    lista_nummer = ser_cont.split(',')
    print(lista_nummer)
    for i in range(0, len(lista_nummer) - 1, 2):
        continent_list.append((int(lista_nummer[i]), int(lista_nummer[i + 1])))
    return continent_list

lista = [(5,5), (6,6), (7,7)] # Testa funktionerna
string_lista = stringify(lista)
print(string_lista)
print(unstringify(string_lista))


5,5,6,6,7,7,
['5', '5', '6', '6', '7', '7', '']
[(5, 5), (6, 6), (7, 7)]


Extrahera kontinenter, karta och byar och serialisera kontinent_listorna.

In [ ]:
kontinenter = ny_karta.get_continents()
